In [169]:
import pandas as pd

In [170]:
pd.options.display.max_columns = None

In [171]:
column_names = ['user_id', 'id', 'count_months', 'amount', 'month_payment',
       'payed_this_month', 'to_pay', 'planned', 'created_at', 'date']
train = pd.DataFrame(columns=column_names)

In [172]:
train.head()

,user_id,id,count_months,amount,month_payment,payed_this_month,to_pay,planned,created_at,date


In [173]:
import mysql.connector
from dateutil.relativedelta import relativedelta
from datetime import date
from decimal import Decimal

prediction_month = '2020-04-01'
pred_month = datetime.strptime(prediction_month, "%Y-%m-%d").date()

conn = mysql.connector.connect(host='localhost', 
                port=3306, user='root', passwd='dalekini', 
                db='digiu')

a = prediction_month
while (a>'2020-01-01'):
    b = (datetime.strptime(a, "%Y-%m-%d").date() + relativedelta(months=-1)).strftime("%Y-%m-%d")
    train_query_string = "SELECT distinct cr.user_id as user_id, cr.id as id, cr.count_months as count_months, cr.amount as amount, cr.month_payment as month_payment,inv.amount as payed_this_month, cps.left_amount as to_pay, cps.amount as planned, cr.created_at as created_at, cps.date as date FROM credit cr join credit_payment_schedule cps on cr.id = cps.credit_id left join (select credit_id, sum(amount)as amount from invoice where created_at > '" + b + "' AND created_at < '" + a + "' group by credit_id) inv on cr.id=inv.credit_id WHERE (cr.created_at > '2019-12-01' AND cr.created_at < '" + b + "') and (cps.date > '" + b + "' AND cps.date < '" + a + "' ) and cr.status in (2) and (cr.user_id > 19) and cr.user_id not in (23,24,26,28,33)"
    train_query = pd.read_sql_query(train_query_string, conn)
    train = train.append(train_query)
    a = (datetime.strptime(a, "%Y-%m-%d").date() + relativedelta(months=-1)).strftime("%Y-%m-%d")
train = train.fillna(0)

a = prediction_month
b = (datetime.strptime(a, "%Y-%m-%d").date() + relativedelta(months=+1)).strftime("%Y-%m-%d")
test_query_string = "SELECT distinct cr.user_id as user_id, cr.id as id, cr.count_months as count_months, cr.amount as amount, cr.month_payment as month_payment,inv.amount as payed_this_month, cps.left_amount as to_pay, cps.amount as planned, cr.created_at as created_at, cps.date as date FROM credit cr join credit_payment_schedule cps on cr.id = cps.credit_id left join (select credit_id, sum(amount)as amount from invoice where created_at > '" + a + "' AND created_at < '" + b + "' group by credit_id) inv on cr.id=inv.credit_id WHERE (cr.created_at > '2019-12-01' AND cr.created_at < '" + a + "') and (cps.date > '" + a + "' AND cps.date < '" + b + "' ) and cr.status in (2) and (cr.user_id > 19) and cr.user_id not in (23,24,26,28,33)"
test = pd.read_sql_query(test_query_string, conn)
test = test.fillna(0)

In [174]:
train.head()

,user_id,id,count_months,amount,month_payment,payed_this_month,to_pay,planned,created_at,date
0,196,6,6,1000.0,100.0,100.0,0.0,100.0,2019-12-18 23:55:33,2020-03-18
1,227,20,7,500.0,50.0,2.0,48.0,50.0,2019-12-20 05:40:39,2020-03-19
2,238,22,7,500.0,50.0,0.0,50.0,50.0,2019-12-20 10:45:33,2020-03-20
3,45,24,26,5000.0,167.0,0.0,0.0,167.0,2019-12-20 14:59:29,2020-03-20
4,123,26,28,5000.0,167.0,0.0,167.0,167.0,2019-12-20 16:56:37,2020-03-20


In [175]:
train.shape

(1087, 10)

In [176]:
test.head()

,user_id,id,count_months,amount,month_payment,payed_this_month,to_pay,planned,created_at,date
0,196,6,6,1000.0,100.0,0.0,100.0,100.0,2019-12-18 23:55:33,2020-04-18
1,227,20,7,500.0,50.0,0.0,50.0,50.0,2019-12-20 05:40:39,2020-04-19
2,238,22,7,500.0,50.0,0.0,50.0,50.0,2019-12-20 10:45:33,2020-04-20
3,45,24,26,5000.0,167.0,0.0,167.0,167.0,2019-12-20 14:59:29,2020-04-20
4,123,26,28,5000.0,167.0,0.0,167.0,167.0,2019-12-20 16:56:37,2020-04-20


In [177]:
test.shape

(1038, 10)

In [178]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, r2_score
from catboost import CatBoostRegressor
import seaborn as sns
import matplotlib.pyplot as plt
features = ['count_months', 'amount', 'month_payment']

X_train = train[features]
X_val = test[features]
y_train = train.payed_this_month.astype('int64').copy()
y_val = test.payed_this_month.astype('int64').copy()

model=CatBoostRegressor(iterations=200, depth=5, learning_rate=0.1, loss_function='RMSE')
model.fit(X_train, y_train)
preds = model.predict(X_val)
print(sum(preds))
pred_catboost = sum(preds)

0:	learn: 51.3830717	total: 1ms	remaining: 200ms
1:	learn: 49.3787463	total: 1.94ms	remaining: 192ms
2:	learn: 47.8615872	total: 3.26ms	remaining: 214ms
3:	learn: 46.5705717	total: 4.25ms	remaining: 208ms
4:	learn: 45.4106750	total: 5.46ms	remaining: 213ms
5:	learn: 44.7374980	total: 7.21ms	remaining: 233ms
6:	learn: 43.6734264	total: 8.23ms	remaining: 227ms
7:	learn: 42.6958040	total: 8.94ms	remaining: 214ms
8:	learn: 41.8493627	total: 9.41ms	remaining: 200ms
9:	learn: 41.1932908	total: 9.85ms	remaining: 187ms
10:	learn: 40.6585502	total: 10.3ms	remaining: 177ms
11:	learn: 40.1944842	total: 10.8ms	remaining: 168ms
12:	learn: 39.7657498	total: 11.2ms	remaining: 161ms
13:	learn: 39.2748041	total: 11.6ms	remaining: 154ms
14:	learn: 38.9485927	total: 12.5ms	remaining: 154ms
15:	learn: 38.5399312	total: 13.1ms	remaining: 151ms
16:	learn: 38.2381834	total: 13.9ms	remaining: 150ms
17:	learn: 38.0162390	total: 14.8ms	remaining: 150ms
18:	learn: 37.7386289	total: 15.3ms	remaining: 145ms
19:	le

In [179]:
from sklearn import ensemble
clf = ensemble.RandomForestClassifier(n_estimators=1000)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_val)

print(sum(y_pred))
pred_randforest = sum(y_pred)

25017


In [180]:
#Предскажем деревом решений

In [181]:
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier()
model.fit(X_train, y_train)
proba = model.predict(X_val)

print( sum(proba))
pred_destree = sum(proba)

23544


In [182]:
Planned = sum(test.planned)
print(Planned)

Payed = sum(test.payed_this_month)
print(Payed)

left = sum((test.to_pay).astype(float))
print(left)

prediction = (pred_catboost+pred_destree+pred_randforest)/3
print(prediction)

46885.0
1010.29
36538.5
24521.51312700647


In [194]:
conn = mysql.connector.connect(host='localhost', 
                port=3306, user='root', passwd='dalekini', 
                db='digiu')
query = "INSERT INTO credit_predictions(month, planned, left_to_pay, prediction) VALUES(%s,%s,%s,%s)"
args = (prediction_month[:-3], Decimal(Planned), Decimal(left), Decimal(prediction))

cursor = conn.cursor()
cursor.execute(query, args)
conn.commit()